In [1]:
import pandas as pd
import numpy as np
import os
import re
import contractions
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import opinion_lexicon

import statsmodels.formula.api as smf
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

# event_study

In [2]:
import pandas_datareader
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
estimation_startdate = datetime(2020,6,11)
estimation_enddate = datetime(2020,12,8)

# 'FTSE 100', 'NASDAQ-100', 'S&P 500', 'CAC 40', 'DAX',
#           'Nikkei 225','Hang Seng Index',  'Shanghai Composite Index', 'S&P/ASX 200', 'S&P BSE SENSEX ',
#           'S&P/TSX Composite Index', 'STI Index', 'ESTX 50 PR.EUR','Russell 2000', 'FTSE MIB Index',
#           'S&P/ASX 50', 'Nifty 50', 'OMX Stockholm 30', 'PSI 20', 'Dow Jones Industrial Average',
#           'iShares MSCI World Index ETF', 'KOSPI Composite Index', 'OMX Copenhagen 25', 'TSEC weighted index', 
#            'S&P/NZX 50 INDEX GROSS', 'STXE 600 PR.EUR','BIST 100',
#            'iShares MSCI World Small Cap UCITS ETF ', 'MSCI AC Asia Pacific EX Japan Index FAM Fund Class I EUR Accumulation ',
#             'Vanguard FTSE Developed Europe ex-U.K. Equity Index Fund GBP Acc'

tickers = ['IMOEX.ME','^FTSE','^FCHI','DAX', 'ENY=F', '^HSI', '000001.SS','^AXJO', '^BSESN', 
           '^GSPTSE', '^STI', '^STOXX50E', '^RUT', 'FTSEMIB.MI', '^AFLI', '^NSEI','^OMX',
           'PSI20.LS', '^DJI', 'XWD.TO','^KS11','^OMXC25','^TWII','^NZ50', '^STOXX',
           'XU100.IS', 'WSML.L', '0P0001GY56.F', 'IEUR',  '0P0000KSP8.L']
marketPrices = pdr.get_data_yahoo(tickers, start = estimation_startdate, end = estimation_enddate)["Adj Close"]

[*********************100%***********************]  30 of 30 completed


In [4]:
marketPrices.head(10)

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1343.319946,NaN,NaN,NaN,NaN,NaN
2020-06-11,2920.896973,25953.519531,98.962997,22.885277,21710.0,18807.0,37.882664,2743.800049,4356.080078,4.75550,...,2176.780029,9902.000000,11154.929688,1609.060059,1318.709961,1356.219971,2704.209961,353.070007,3144.570068,11535.769531
2020-06-12,2919.740967,25863.500000,99.253998,23.111237,22150.0,18888.0,38.585365,NaN,4359.990234,4.69650,...,2132.300049,9972.900391,10905.940430,1609.069946,1320.500000,1387.680054,2684.629883,354.059998,3153.739990,11429.940430
2020-06-15,2890.030029,25829.519531,96.971001,23.428486,21920.0,18969.0,38.946232,2719.060059,4326.779785,4.73825,...,2030.819946,9813.700195,10864.120117,1616.140015,1316.959961,1419.609985,2613.879883,353.089996,3136.399902,11306.259766
2020-06-16,2931.750000,26459.050781,99.992996,23.870466,22350.0,19626.0,39.384750,2748.300049,4432.270020,4.89100,...,2138.050049,9914.000000,10953.849609,1671.839966,1336.680054,1452.260010,2666.850098,363.329987,3242.649902,11511.639648
2020-06-17,2935.870117,26730.359375,100.750000,23.703255,22460.0,19586.0,39.457840,2740.100098,4450.470215,4.87825,...,2141.050049,9881.150391,11334.309570,1677.530029,1357.670044,1426.530029,2669.620117,366.019989,3267.260010,11534.589844
2020-06-18,2939.320068,26850.769531,100.806999,23.592083,22350.0,19486.0,39.183765,2724.330078,4428.240234,4.86400,...,2133.479980,10091.650391,11225.280273,1667.930054,1347.589966,1427.329956,2665.659912,363.410004,3249.899902,11548.330078
2020-06-19,2967.629883,27014.220703,101.589996,23.377872,22290.0,19619.0,39.019314,2758.669922,4461.270020,4.88525,...,2141.320068,10244.400391,11254.740234,NaN,1360.739990,1418.630005,2634.830078,365.459991,3269.100098,11549.860352
2020-06-22,2965.271973,26782.349609,100.967003,23.716812,22550.0,19479.0,39.457840,2763.290039,4414.200195,4.84250,...,2126.729980,10311.200195,11158.459961,1657.709961,1356.640015,1433.530029,2629.689941,362.700012,3241.689941,11572.929688


In [5]:
marketReturns  = marketPrices.pct_change(1).dropna()
#marketReturns = np.where(int(marketReturns) == 0, )
marketReturns.head()

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-12,-0.000396,-0.003468,0.002940,0.009874,0.020267,0.004307,0.018549,0.000000,0.000898,-0.012407,...,-0.020434,0.007160,-0.022321,0.000006,0.001357,0.023197,-0.007241,0.002804,0.002916,-0.009174
2020-06-15,-0.010176,-0.001314,-0.023002,0.013727,-0.010384,0.004288,0.009352,-0.009017,-0.007617,0.008890,...,-0.047592,-0.015963,-0.003835,0.004394,-0.002681,0.023010,-0.026354,-0.002740,-0.005498,-0.010821
2020-06-16,0.014436,0.024373,0.031164,0.018865,0.019617,0.034635,0.011260,0.010754,0.024381,0.032238,...,0.052801,0.010220,0.008259,0.034465,0.014974,0.022999,0.020265,0.029001,0.033876,0.018165
2020-06-17,0.001405,0.010254,0.007571,-0.007005,0.004922,-0.002038,0.001856,-0.002984,0.004106,-0.002607,...,0.001403,-0.003313,0.034733,0.003403,0.015703,-0.017717,0.001039,0.007404,0.007590,0.001994
2020-06-18,0.001175,0.004505,0.000566,-0.004690,-0.004898,-0.005106,-0.006946,-0.005755,-0.004995,-0.002921,...,-0.003536,0.021303,-0.009619,-0.005723,-0.007425,0.000561,-0.001483,-0.007131,-0.005313,0.001191


In [6]:
marketReturns["whole market"] = marketReturns.mean(axis = 1)
marketReturns.head(10)

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII,whole market
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-12,-0.000396,-0.003468,0.002940,0.009874,0.020267,0.004307,0.018549,0.000000,0.000898,-0.012407,...,0.007160,-0.022321,0.000006,0.001357,0.023197,-0.007241,0.002804,0.002916,-0.009174,0.001454
2020-06-15,-0.010176,-0.001314,-0.023002,0.013727,-0.010384,0.004288,0.009352,-0.009017,-0.007617,0.008890,...,-0.015963,-0.003835,0.004394,-0.002681,0.023010,-0.026354,-0.002740,-0.005498,-0.010821,-0.005647
2020-06-16,0.014436,0.024373,0.031164,0.018865,0.019617,0.034635,0.011260,0.010754,0.024381,0.032238,...,0.010220,0.008259,0.034465,0.014974,0.022999,0.020265,0.029001,0.033876,0.018165,0.022371
2020-06-17,0.001405,0.010254,0.007571,-0.007005,0.004922,-0.002038,0.001856,-0.002984,0.004106,-0.002607,...,-0.003313,0.034733,0.003403,0.015703,-0.017717,0.001039,0.007404,0.007590,0.001994,0.002834
2020-06-18,0.001175,0.004505,0.000566,-0.004690,-0.004898,-0.005106,-0.006946,-0.005755,-0.004995,-0.002921,...,0.021303,-0.009619,-0.005723,-0.007425,0.000561,-0.001483,-0.007131,-0.005313,0.001191,-0.001296
2020-06-19,0.009631,0.006087,0.007767,-0.009080,-0.002685,0.006825,-0.004197,0.012605,0.007459,0.004369,...,0.015136,0.002624,0.000000,0.009758,-0.006095,-0.011566,0.005641,0.005908,0.000133,0.003272
2020-06-22,-0.000795,-0.008583,-0.006132,0.014498,0.011664,-0.007136,0.011239,0.001675,-0.010551,-0.008751,...,0.006521,-0.008555,-0.006127,-0.003013,0.010503,-0.001951,-0.007552,-0.008385,0.001997,-0.000463
2020-06-23,0.001804,0.015380,0.000000,0.016959,-0.001774,0.018635,0.007641,0.010379,0.008038,0.013836,...,0.015498,-0.002319,0.015473,0.016703,0.004053,0.001989,0.012958,0.017627,0.003407,0.008456
2020-06-24,0.003006,-0.025886,0.010251,-0.026794,-0.012439,-0.034220,-0.026425,0.000000,-0.017475,-0.032539,...,-0.015825,0.011393,-0.024029,-0.017545,-0.034460,-0.002391,-0.027844,-0.031135,0.004160,-0.014610


In [7]:
Beta = {}
var = marketReturns["whole market"].var()
for col in marketReturns:
    cov = marketReturns[col].cov(marketReturns["whole market"])
    Beta[col] = cov/var
Beta

{'000001.SS': 0.4747550414318911,
 '0P0000KSP8.L': 1.0853506829642139,
 '0P0001GY56.F': 0.6630363980518439,
 'DAX': 1.3041179646718368,
 'ENY=F': 0.7589514195760492,
 'FTSEMIB.MI': 1.4611080041450082,
 'IEUR': 1.048060859939104,
 'IMOEX.ME': 0.7420710872367234,
 'PSI20.LS': 0.9948225963926122,
 'WSML.L': 1.2063700364580716,
 'XU100.IS': 4.462247329104214,
 'XWD.TO': 0.5970979017673801,
 '^AFLI': 0.5836164984949977,
 '^AXJO': 0.5848643030005657,
 '^BSESN': 0.6864939787379023,
 '^DJI': 0.9125726937706536,
 '^FCHI': 1.4414631245478144,
 '^FTSE': 1.244362999557146,
 '^GSPTSE': 0.5706970952108421,
 '^HSI': 0.8830157277550673,
 '^KS11': 0.75971283751209,
 '^NSEI': 0.6808791770648169,
 '^NZ50': 0.33143443474655826,
 '^OMX': 1.1024619856650082,
 '^OMXC25': 0.6620966322639888,
 '^RUT': 1.1034578898455905,
 '^STI': 0.6450921747936109,
 '^STOXX': 1.1916719768580952,
 '^STOXX50E': 1.4509326788820025,
 '^TWII': 0.3671844695543191,
 'whole market': 0.9999999999999998}

In [8]:
event_startdate = datetime(2020,12,8)
event_enddate = datetime(2020,12,21)
eventPrices = pdr.get_data_yahoo(tickers, start = event_startdate, end = event_enddate)["Adj Close"]
eventPrices

[*********************100%***********************]  30 of 30 completed


,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1563.560059,NaN,NaN,NaN,NaN,NaN
2020-12-08,3410.179932,29593.089844,117.942001,27.565037,26570.0,22053.0,46.592953,3179.610107,4759.109863,6.1870,...,2700.929932,13392.950195,12719.589844,1925.500000,1579.640015,1917.780029,2825.629883,393.640015,3525.870117,14360.400391
2020-12-09,3371.964111,29512.230469,118.750999,27.542179,26690.0,21970.0,46.574680,3211.659912,4774.189941,6.2185,...,2755.469971,13529.099609,12889.400391,1926.420044,1588.939941,1902.150024,2843.070068,394.899994,3529.020020,14390.139648
2020-12-10,3373.275879,29819.330078,117.967003,27.747889,26710.0,21916.0,46.529003,3258.310059,4795.470215,6.1930,...,2746.459961,13478.299805,12860.370117,1906.819946,1576.910034,1922.699951,2824.959961,393.149994,3522.310059,14249.490234
2020-12-11,3347.190918,29703.140625,118.372002,27.281612,26540.0,21702.0,46.227516,3276.580078,4742.669922,6.1760,...,2770.060059,13513.849609,12919.250000,1891.219971,1581.969971,1911.699951,2821.699951,390.119995,3485.840088,14261.690430
2020-12-14,3369.120117,29631.500000,117.723999,27.555893,26660.0,21760.0,46.252434,3254.830078,4780.029785,6.2260,...,2762.199951,13558.150391,12835.120117,1890.650024,1600.270020,1913.859985,2858.139893,391.850006,3503.959961,14211.049805
2020-12-15,3367.229980,29604.140625,117.154999,27.971882,26760.0,21935.0,46.879833,3248.110107,4775.919922,6.2500,...,2756.820068,13567.849609,12767.169922,1889.839966,1595.310059,1959.760010,2856.719971,392.839996,3521.500000,14068.519531
2020-12-16,3366.982910,29676.750000,118.160004,28.393360,26760.0,21987.0,47.119724,3269.949951,4796.080078,6.2950,...,2771.790039,13682.700195,12828.990234,1892.260010,1612.619995,1952.719971,2872.800049,396.079987,3543.000000,14304.459961
2020-12-17,3404.873047,29722.349609,118.403000,28.799294,26750.0,22012.0,47.479553,3282.669922,4825.529785,6.3490,...,2770.429932,13740.700195,12888.769531,1885.180054,1618.410034,1978.050049,2858.020020,397.279999,3560.870117,14258.929688


In [9]:
eventReturns = eventPrices.pct_change(1).dropna()
eventReturns["whole market"] = eventReturns.mean(axis = 1)
eventReturns

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII,whole market
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-09,-0.011206,-0.002732,0.006859,-0.000829,0.004516,-0.003764,-0.000392,0.010080,0.003169,0.005091,...,0.010166,0.013350,0.000478,0.005887,-0.008150,0.006172,0.003201,0.000893,0.002071,0.002987
2020-12-10,0.000389,0.010406,-0.006602,0.007469,0.000749,-0.002458,-0.000981,0.014525,0.004457,-0.004101,...,-0.003755,-0.002252,-0.010174,-0.007571,0.010804,-0.006370,-0.004432,-0.001901,-0.009774,-0.001083
2020-12-11,-0.007733,-0.003896,0.003433,-0.016804,-0.006365,-0.009765,-0.006480,0.005607,-0.011010,-0.002745,...,0.002638,0.004578,-0.008181,0.003209,-0.005721,-0.001154,-0.007707,-0.010354,0.000856,-0.002674
2020-12-14,0.006552,-0.002412,-0.005474,0.010054,0.004521,0.002673,0.000539,-0.006638,0.007877,0.008096,...,0.003278,-0.006512,-0.000301,0.011568,0.001130,0.012914,0.004435,0.005198,-0.003551,0.001635
2020-12-15,-0.000561,-0.000923,-0.004833,0.015096,0.003751,0.008042,0.013565,-0.002065,-0.000860,0.003855,...,0.000715,-0.005294,-0.000428,-0.003099,0.023983,-0.000497,0.002526,0.005006,-0.010030,0.001972
2020-12-16,-0.000073,0.002453,0.008578,0.015068,0.000000,0.002371,0.005117,0.006724,0.004221,0.007200,...,0.008465,0.004842,0.001281,0.010851,-0.003592,0.005629,0.008248,0.006105,0.016771,0.005488
2020-12-17,0.011253,0.001537,0.002057,0.014297,-0.000374,0.001137,0.007636,0.003890,0.006140,0.008578,...,0.004239,0.004660,-0.003742,0.003590,0.012972,-0.005145,0.003030,0.005044,-0.003183,0.004300
2020-12-18,-0.002929,0.005996,-0.004020,-0.007143,-0.001495,-0.001635,-0.004664,-0.002717,-0.012952,0.003623,...,0.001445,-0.016043,0.000902,0.015713,-0.004075,-0.003163,-0.003474,-0.004249,-0.000629,-0.002712


In [10]:
non_risk_insterest_rate = 0.0384

abnormalReturns = pd.DataFrame()

for col in marketReturns:
    abnormalReturns[col] = non_risk_insterest_rate + Beta[col]*(eventReturns[col] - non_risk_insterest_rate) - eventReturns[col]

abnormalReturns

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII,whole market
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-09,0.026056,-0.003511,0.010628,-0.011930,0.008168,-0.019442,-0.001864,0.007305,0.000182,-0.006874,...,0.009010,0.016747,-0.003886,0.010986,-0.004816,0.011438,-0.006747,-0.016913,0.022990,8.239937e-18
2020-12-10,0.019965,-0.002389,0.015164,-0.009407,0.009076,-0.018840,-0.001893,0.006158,0.000176,-0.008771,...,0.013452,0.027179,-0.004977,0.015534,-0.002855,0.015889,-0.008210,-0.018173,0.030485,4.119968e-18
2020-12-11,0.024231,-0.003610,0.011783,-0.016789,0.010790,-0.022209,-0.002157,0.008458,0.000256,-0.008491,...,0.011413,0.022612,-0.004773,0.011891,-0.004565,0.014038,-0.008837,-0.021985,0.023758,5.204170e-18
2020-12-14,0.016728,-0.003483,0.014784,-0.008621,0.008166,-0.016474,-0.001820,0.011617,0.000158,-0.006254,...,0.011208,0.030027,-0.003965,0.009067,-0.003856,0.009045,-0.006510,-0.014972,0.026547,9.324139e-18
2020-12-15,0.020464,-0.003356,0.014568,-0.007087,0.008352,-0.013998,-0.001194,0.010437,0.000203,-0.007129,...,0.012026,0.029212,-0.003978,0.014023,-0.001492,0.013805,-0.006876,-0.015059,0.030647,3.903128e-18
2020-12-16,0.020208,-0.003068,0.010049,-0.007096,0.009256,-0.016613,-0.001600,0.008170,0.000177,-0.006439,...,0.009553,0.022436,-0.003803,0.009309,-0.004344,0.011631,-0.005779,-0.014563,0.013687,5.204170e-18
2020-12-17,0.014259,-0.003146,0.012246,-0.007330,0.009346,-0.017182,-0.001479,0.008901,0.000167,-0.006154,...,0.010902,0.022558,-0.004318,0.011762,-0.002631,0.015454,-0.006779,-0.015041,0.026314,9.540979e-18
2020-12-18,0.021708,-0.002766,0.014294,-0.013850,0.009617,-0.018461,-0.002070,0.010605,0.000266,-0.007177,...,0.011793,0.036398,-0.003842,0.007666,-0.004394,0.014751,-0.008026,-0.019232,0.024698,6.938894e-18


In [11]:
all_AR = pd.DataFrame(abnormalReturns.mean(axis = 1)/len(tickers))
all_AR.columns = ['AAR']
all_AR

,AAR
Date,
2020-12-09,-0.000003
2020-12-10,0.000032
2020-12-11,0.000023
2020-12-14,0.000060
2020-12-15,0.000045
2020-12-16,-0.000035
2020-12-17,0.000002
2020-12-18,0.000041


In [12]:
CAR_list = [all_AR.iloc[0]["AAR"]]
for i in range(1, len(all_AR)):
    CAR_list.append(all_AR.iloc[i-1]["AAR"]+all_AR.iloc[i]["AAR"])
all_AR["CAR"] = CAR_list
all_AR

,AAR,CAR
Date,,
2020-12-09,-0.000003,-0.000003
2020-12-10,0.000032,0.000029
2020-12-11,0.000023,0.000056
2020-12-14,0.000060,0.000084
2020-12-15,0.000045,0.000105
2020-12-16,-0.000035,0.000009
2020-12-17,0.000002,-0.000033
2020-12-18,0.000041,0.000044


# cross_sectional

In [13]:
path_name = "L:/year2_sem3/Pembroke/supervision/US presidential selection 2020"
NEWS_FILE1 = os.listdir(f"{path_name}/FT")
NEWS_FILE2 = os.listdir(f"{path_name}/TWSJ")
NEWS_FILE = [file for file in NEWS_FILE1]+[file for file in NEWS_FILE2]
NEWS_FILE

['A changed US Senate awaits Joe Biden_2020-12-14.txt',
 'Activist CEOs are focusing on the bottom line_2020-12-16.txt',
 'America’s PPE supply chain looks distinctly sick_2020-12-17.txt',
 'Asean trade deal emerging from the shadows_2020-12-15.txt',
 'Australia’s ‘amateurish’ China diplomacy sets business on edge_2020-12-17.txt',
 'Barr to step down as US attorney-general after Trump attacks_2020-12-15.txt',
 'Biden’s ‘Obama reunion’ cabinet_2022-12-11.txt',
 'Boom conditions for US homebuilders defy the pandemic_2020-12-18.txt',
 'Brazil’s price tag on its climate goals scorned by environmentalists_2020-12-10.txt',
 'Breaking down BP’s bet on carbon credits_2022-12-18.txt',
 'Britain’s annual Brexit capitulation draws nigh_2020-12-17.txt',
 'Brussels warns on corruption threat to taxpayer trust_2020-12-15.txt',
 'China pulls back from the world rethinking Xi’s ‘project of the century’_2020-12-11.txt',
 'China’s power is rising in an unstable world_2020-12-16.txt',
 'Climate leaders p

In [14]:
len(NEWS_FILE)

210

# Text Normalization

In [15]:
stop_words = set(stopwords.words('english'))
stop_words.discard('not')

In [16]:
def normalize(text, default_replace = ""):
    #handle URL
    text = re.sub("(http|https):\/\/\S+", default_replace, text)
    
    #remove upper capitalization
    text = text.lower()
    
    #word contraction
    for k,v in contractions.contractions_dict.items():
        text = text.replace(k,v)
     
    #tokenization
    token_list = word_tokenize(text)
    
    token_list = [token for token in token_list if token not in string.punctuation]
    token_list = [token for token in token_list if token.isalpha()]
    token_list = [token for token in token_list if not token in stop_words]

    return token_list

# Stemming

In [17]:
stemmer = SnowballStemmer("english")

In [18]:
def stem_tokens(tokens, stemmer):
    token_list = []
    for token in tokens:
        token_list.append(stemmer.stem(token))
    return token_list

In [19]:
def process_text(file):
    folder = ""
    if file in NEWS_FILE1:
        folder = "FT"
    elif file in NEWS_FILE2:
        folder = "TWSJ"
    with open(f'{path_name}/{folder}/{file}', encoding="utf8") as f:
        text = f.read()
    
    tokens = normalize(text)
    stem = stem_tokens(tokens, stemmer)

    return stem   

# Sentiment analysis 

In [20]:
positive_lexiceon = set(opinion_lexicon.positive())
negative_lexiceon = set(opinion_lexicon.negative())

In [21]:
sentiment_scores = {}
for i in range(len(NEWS_FILE)):
    date = os.path.splitext(NEWS_FILE[i])[0].split('_')[-1].split('-')[-1]
    
    cleaned_words = process_text(NEWS_FILE[i])
    if len(cleaned_words) == 0:
        print(NEWS_FILE[i])
    positive_sentiment = 0
    negative_sentiment = 0
    for word in cleaned_words:       
        if word in positive_lexiceon:
            positive_sentiment += 1
        elif word in negative_lexiceon:
            negative_sentiment += 1
        
    phi_pos = positive_sentiment / len(cleaned_words)
    phi_neg = negative_sentiment / len(cleaned_words)
        
    phi_npt = (phi_pos - phi_neg) / (phi_pos + phi_neg)
        
    sentiment_scores[NEWS_FILE[i]] = [phi_pos, phi_neg, phi_npt]

sentiment_scores

{'A changed US Senate awaits Joe Biden_2020-12-14.txt': [0.0199501246882793,
  0.03740648379052369,
  -0.30434782608695654],
 'Activist CEOs are focusing on the bottom line_2020-12-16.txt': [0.06466512702078522,
  0.025404157043879907,
  0.4358974358974359],
 'America’s PPE supply chain looks distinctly sick_2020-12-17.txt': [0.0321285140562249,
  0.050200803212851405,
  -0.21951219512195125],
 'Asean trade deal emerging from the shadows_2020-12-15.txt': [0.0392156862745098,
  0.025374855824682813,
  0.2142857142857143],
 'Australia’s ‘amateurish’ China diplomacy sets business on edge_2020-12-17.txt': [0.028148148148148148,
  0.05037037037037037,
  -0.28301886792452835],
 'Barr to step down as US attorney-general after Trump attacks_2020-12-15.txt': [0.0671462829736211,
  0.02877697841726619,
  0.39999999999999997],
 'Biden’s ‘Obama reunion’ cabinet_2022-12-11.txt': [0.06277056277056277,
  0.04220779220779221,
  0.19587628865979378],
 'Boom conditions for US homebuilders defy the pande

In [22]:
sentiment = pd.DataFrame(sentiment_scores, index = ['positive','negative','npt_positive'] )
sentiment

,A changed US Senate awaits Joe Biden_2020-12-14.txt,Activist CEOs are focusing on the bottom line_2020-12-16.txt,America’s PPE supply chain looks distinctly sick_2020-12-17.txt,Asean trade deal emerging from the shadows_2020-12-15.txt,Australia’s ‘amateurish’ China diplomacy sets business on edge_2020-12-17.txt,Barr to step down as US attorney-general after Trump attacks_2020-12-15.txt,Biden’s ‘Obama reunion’ cabinet_2022-12-11.txt,Boom conditions for US homebuilders defy the pandemic_2020-12-18.txt,Brazil’s price tag on its climate goals scorned by environmentalists_2020-12-10.txt,Breaking down BP’s bet on carbon credits_2022-12-18.txt,...,When the Stock Market Is Too Much Fun_2020-12-11.txt,"Who Is Jeffrey Rosen, Who Will Succeed Attorney General William Barr_2020-12-14.txt",William Barr to Resign as Attorney General_2020-12-15.txt,William Barr ‘One Standard of Justice’_2020-12-18.txt,Wisconsin Supreme Court Denies Trump Campaign Effort to Throw Out Votes_2020-12-14.txt,World Bank Says Managers Pressured Staff to Alter Global Business Rankings_2020-12-17.txt,"‘Doomscrolling’ A Twitter Habit Is the New, High-Tech Way to Slide Into Despair_2020-12-10.txt",‘Intimations’ Review Scenes from a Global Humbling_2020-12-18.txt,‘The Nutcracker’ Reviews Balanchine at Home for the Holidays_2020-12-16.txt,‘The Stand’ Review A Dystopia for Today and Tomorrow_2020-12-15.txt
positive,0.019950,0.064665,0.032129,0.039216,0.028148,0.067146,0.062771,0.041763,0.026699,0.038889,...,0.030628,0.042644,0.062937,0.052372,0.053309,0.014124,0.034667,0.042424,0.053933,0.031579
negative,0.037406,0.025404,0.050201,0.025375,0.050370,0.028777,0.042208,0.020882,0.016990,0.034444,...,0.035222,0.027719,0.029138,0.037549,0.034926,0.036723,0.021333,0.074747,0.015730,0.052632
npt_positive,-0.304348,0.435897,-0.219512,0.214286,-0.283019,0.400000,0.195876,0.333333,0.222222,0.060606,...,-0.069767,0.212121,0.367089,0.164835,0.208333,-0.444444,0.238095,-0.275862,0.548387,-0.250000


In [23]:
def sentimentCount(lst,date,npt):
    if date == '10':
        lst[0] += npt
    if date == '11':
        lst[1] += npt
    elif date == '14':
        lst[2] += npt
    elif date == '15':
        lst[3] += npt
    elif date == '16':
        lst[4] += npt
    elif date == '17':
        lst[5] += npt
    elif date == '18':
        lst[6] += npt
    return lst

In [24]:
sentiment_list = [0 for i in range(7)]
for col in sentiment:
    date =  os.path.splitext(col)[0].split('_')[-1].split('-')[-1]
    sentiment_list = sentimentCount(sentiment_list,date,sentiment[col].values[2])
sentiment_list = [i/10 for i in sentiment_list]
sentiment_list

[0.04846159165455346,
 0.3365681511165658,
 0.06179325609031119,
 0.4502387301810672,
 0.4047826832464775,
 -0.049462650112095954,
 -0.061663025712260046]

# Size

In [25]:
import pandas_datareader
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

from datetime import datetime
import matplotlib.pyplot as plt

In [26]:
size_startdate = datetime(2020,12,10)
size_enddate = datetime(2020,12,21)

# 'FTSE 100', 'NASDAQ-100', 'S&P 500', 'CAC 40', 'DAX',
#           'Nikkei 225','Hang Seng Index',  'Shanghai Composite Index', 'S&P/ASX 200', 'S&P BSE SENSEX ',
#           'S&P/TSX Composite Index', 'STI Index', 'ESTX 50 PR.EUR','Russell 2000', 'FTSE MIB Index',
#           'S&P/ASX 50', 'Nifty 50', 'OMX Stockholm 30', 'PSI 20', 'Dow Jones Industrial Average',
#           'iShares MSCI World Index ETF', 'KOSPI Composite Index', 'OMX Copenhagen 25', 'TSEC weighted index', 
#            'S&P/NZX 50 INDEX GROSS', 'STXE 600 PR.EUR','BIST 100',
#            'iShares MSCI World Small Cap UCITS ETF ', 'MSCI AC Asia Pacific EX Japan Index FAM Fund Class I EUR Accumulation ',
#             'Vanguard FTSE Developed Europe ex-U.K. Equity Index Fund GBP Acc'

tickers = ['IMOEX.ME','^FTSE','^FCHI','DAX', 'ENY=F', '^HSI', '000001.SS','^AXJO', '^BSESN', 
           '^GSPTSE', '^STI', '^STOXX50E', '^RUT', 'FTSEMIB.MI', '^AFLI', '^NSEI','^OMX',
           'PSI20.LS', '^DJI', 'XWD.TO','^KS11','^OMXC25','^TWII','^NZ50', '^STOXX',
           'XU100.IS', 'WSML.L', '0P0001GY56.F', 'IEUR',  '0P0000KSP8.L']
volumes = pdr.get_data_yahoo(tickers, start = size_startdate, end = size_enddate)["Volume"][1:]

[*********************100%***********************]  30 of 30 completed


In [27]:
volumes

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-10,247300.0,0.0,0.0,12600.0,0.0,450860100.0,216200.0,0.0,91298600.0,103410.0,...,1227700.0,553400.0,0.0,0.0,14776300,4.658480e+09,333477300.0,253403800.0,38432300.0,6332100.0
2020-12-11,298600.0,0.0,0.0,10500.0,0.0,428052900.0,177000.0,0.0,109334400.0,103735.0,...,1215200.0,787700.0,44343000.0,0.0,13873000,4.375470e+09,290440600.0,244825600.0,35371700.0,6398300.0
2020-12-14,239800.0,0.0,0.0,3400.0,0.0,389977700.0,2112900.0,0.0,102523600.0,3047169.0,...,1113600.0,558400.0,0.0,75682300.0,13352800,4.623850e+09,302651900.0,207740500.0,32494200.0,5733400.0
2020-12-15,225700.0,0.0,0.0,15100.0,0.0,347534000.0,287600.0,0.0,83439700.0,360520.0,...,1092700.0,497600.0,0.0,121123000.0,15657300,4.387080e+09,276181700.0,204633700.0,32146400.0,6314600.0
2020-12-16,220800.0,0.0,0.0,30600.0,0.0,447187000.0,461400.0,0.0,74654400.0,718026.0,...,1166600.0,462200.0,0.0,0.0,16657900,4.067040e+09,291312500.0,227325300.0,38833500.0,4861300.0
2020-12-17,275500.0,0.0,0.0,15800.0,0.0,456361000.0,456600.0,0.0,48743200.0,329898.0,...,1015500.0,416000.0,0.0,0.0,15781200,4.192810e+09,284079500.0,207353400.0,32518600.0,4300900.0
2020-12-18,281000.0,0.0,0.0,14700.0,0.0,653820900.0,509400.0,0.0,67068000.0,212982.0,...,1058300.0,519400.0,84442800.0,0.0,20941100,7.097100e+09,337801200.0,347207600.0,69147500.0,5686900.0


In [28]:
#volumes.drop(columns = ["^OMX"], inplace = True)
for col in volumes:
    if int(volumes[col].mean()) == 0:
        volumes.drop(columns = [col], inplace = True)
        continue
    for i in range(len(volumes)):
        if pd.isnull(volumes[col].iloc[i]):
            volumes[col].iloc[i] = volumes[col].iloc[i-1]
    
volumes

,000001.SS,DAX,FTSEMIB.MI,IEUR,PSI20.LS,WSML.L,XU100.IS,XWD.TO,^AFLI,^AXJO,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-10,247300.0,12600.0,450860100.0,216200.0,91298600.0,103410.0,5.896164e+09,15369.0,293200.0,705900.0,...,1227700.0,553400.0,0.0,0.0,14776300,4.658480e+09,333477300.0,253403800.0,38432300.0,6332100.0
2020-12-11,298600.0,10500.0,428052900.0,177000.0,109334400.0,103735.0,5.148071e+09,11677.0,290300.0,688000.0,...,1215200.0,787700.0,44343000.0,0.0,13873000,4.375470e+09,290440600.0,244825600.0,35371700.0,6398300.0
2020-12-14,239800.0,3400.0,389977700.0,2112900.0,102523600.0,3047169.0,4.631732e+09,15070.0,228700.0,575600.0,...,1113600.0,558400.0,0.0,75682300.0,13352800,4.623850e+09,302651900.0,207740500.0,32494200.0,5733400.0
2020-12-15,225700.0,15100.0,347534000.0,287600.0,83439700.0,360520.0,5.279954e+09,5589.0,252500.0,626200.0,...,1092700.0,497600.0,0.0,121123000.0,15657300,4.387080e+09,276181700.0,204633700.0,32146400.0,6314600.0
2020-12-16,220800.0,30600.0,447187000.0,461400.0,74654400.0,718026.0,5.024839e+09,18463.0,236500.0,652500.0,...,1166600.0,462200.0,0.0,0.0,16657900,4.067040e+09,291312500.0,227325300.0,38833500.0,4861300.0
2020-12-17,275500.0,15800.0,456361000.0,456600.0,48743200.0,329898.0,4.448045e+09,9681.0,326200.0,801300.0,...,1015500.0,416000.0,0.0,0.0,15781200,4.192810e+09,284079500.0,207353400.0,32518600.0,4300900.0
2020-12-18,281000.0,14700.0,653820900.0,509400.0,67068000.0,212982.0,4.457055e+09,8782.0,583300.0,1437000.0,...,1058300.0,519400.0,84442800.0,0.0,20941100,7.097100e+09,337801200.0,347207600.0,69147500.0,5686900.0


In [29]:
volumes['size'] = np.log(volumes.mean(axis = 1)) / 100
volumes

,000001.SS,DAX,FTSEMIB.MI,IEUR,PSI20.LS,WSML.L,XU100.IS,XWD.TO,^AFLI,^AXJO,...,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII,size
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-10,247300.0,12600.0,450860100.0,216200.0,91298600.0,103410.0,5.896164e+09,15369.0,293200.0,705900.0,...,553400.0,0.0,0.0,14776300,4.658480e+09,333477300.0,253403800.0,38432300.0,6332100.0,0.202008
2020-12-11,298600.0,10500.0,428052900.0,177000.0,109334400.0,103735.0,5.148071e+09,11677.0,290300.0,688000.0,...,787700.0,44343000.0,0.0,13873000,4.375470e+09,290440600.0,244825600.0,35371700.0,6398300.0,0.201714
2020-12-14,239800.0,3400.0,389977700.0,2112900.0,102523600.0,3047169.0,4.631732e+09,15070.0,228700.0,575600.0,...,558400.0,0.0,75682300.0,13352800,4.623850e+09,302651900.0,207740500.0,32494200.0,5733400.0,0.201145
2020-12-15,225700.0,15100.0,347534000.0,287600.0,83439700.0,360520.0,5.279954e+09,5589.0,252500.0,626200.0,...,497600.0,0.0,121123000.0,15657300,4.387080e+09,276181700.0,204633700.0,32146400.0,6314600.0,0.201545
2020-12-16,220800.0,30600.0,447187000.0,461400.0,74654400.0,718026.0,5.024839e+09,18463.0,236500.0,652500.0,...,462200.0,0.0,0.0,16657900,4.067040e+09,291312500.0,227325300.0,38833500.0,4861300.0,0.200827
2020-12-17,275500.0,15800.0,456361000.0,456600.0,48743200.0,329898.0,4.448045e+09,9681.0,326200.0,801300.0,...,416000.0,0.0,0.0,15781200,4.192810e+09,284079500.0,207353400.0,32518600.0,4300900.0,0.200510
2020-12-18,281000.0,14700.0,653820900.0,509400.0,67068000.0,212982.0,4.457055e+09,8782.0,583300.0,1437000.0,...,519400.0,84442800.0,0.0,20941100,7.097100e+09,337801200.0,347207600.0,69147500.0,5686900.0,0.204025


# risk

In [30]:
risk_startdate = datetime(2020,11,25)
risk_enddate = datetime(2020,12,21)
riskPrices = pdr.get_data_yahoo(tickers, start = risk_startdate, end = risk_enddate)["Adj Close"]
riskPrices

[*********************100%***********************]  30 of 30 completed


,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1558.890015,NaN,NaN,NaN,NaN,NaN
2020-11-25,3362.326904,28917.550781,117.750999,27.107904,26310.0,22304.0,45.588009,3134.520020,4628.629883,6.0000,...,2601.540039,12858.400391,12678.759766,1931.430054,1562.859985,1845.020020,2869.550049,392.089996,3511.899902,13738.830078
2020-11-26,3369.732910,29056.849609,118.517998,NaN,NaN,22201.0,NaN,3138.620117,4607.250000,6.0045,...,2625.909912,12987.000000,12602.019531,1931.969971,NaN,NaN,2857.479980,391.630005,3510.939941,13845.660156
2020-11-27,3408.306885,29309.609375,118.232002,27.361155,26780.0,22352.0,45.743317,3142.679932,4651.819824,6.0400,...,2633.449951,12968.950195,12639.830078,1937.089966,1593.180054,1855.270020,2855.820068,393.230011,3527.790039,13867.089844
2020-11-30,3391.760010,29079.679688,115.612000,27.141733,26310.0,22061.0,45.085533,3107.580078,4604.720215,5.9740,...,2591.340088,NaN,12768.519531,1917.540039,1596.780029,1819.819946,2805.949951,389.359985,3492.540039,13722.889648
2020-12-01,3451.939941,29349.810547,116.339996,27.629034,26920.0,22100.0,46.163563,3147.790039,4587.609863,6.0450,...,2634.250000,13109.049805,12729.910156,1924.000000,1581.619995,1836.050049,2814.120117,391.899994,3525.239990,13885.669922
2020-12-02,3449.381104,29499.740234,116.213997,27.606180,26830.0,21972.0,46.190975,3189.610107,4624.569824,6.0360,...,2675.899902,13113.750000,12728.690430,1918.989990,1575.689941,1838.030029,2810.949951,391.690002,3521.320068,13989.139648
2020-12-03,3442.135986,29227.289062,116.417000,27.565037,26710.0,22007.0,46.337151,3162.669922,4639.140137,6.1145,...,2696.219971,13133.900391,12648.910156,1912.180054,1554.040039,1848.699951,2822.340088,391.720001,3517.100098,13977.089844
2020-12-04,3444.581055,29402.550781,117.560997,27.619892,26860.0,22179.0,46.748260,3184.719971,4702.819824,6.1670,...,2731.449951,13258.549805,12631.379883,1916.939941,1554.979980,1892.449951,2839.889893,394.040009,3539.270020,14132.440430


In [31]:
riskReturns  = riskPrices.pct_change(1).dropna()[1:]
#marketReturns = np.where(int(marketReturns) == 0, )
riskReturns

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-27,0.011447,0.008699,-0.002413,0.009342,0.017864,0.006801,0.003407,0.001294,0.009674,0.005912,...,0.002871,-0.001390,0.003000,0.002650,0.019400,0.005555,-0.000581,0.004086,0.004799,0.001548
2020-11-30,-0.004855,-0.007845,-0.022160,-0.008019,-0.017550,-0.013019,-0.014380,-0.011169,-0.010125,-0.010927,...,-0.015990,0.000000,0.010181,-0.010092,0.002260,-0.019108,-0.017463,-0.009842,-0.009992,-0.010399
2020-12-01,0.017743,0.009289,0.006297,0.017954,0.023185,0.001768,0.023911,0.012939,-0.003716,0.011885,...,0.016559,0.010803,-0.003024,0.003369,-0.009494,0.008919,0.002912,0.006524,0.009363,0.011862
2020-12-02,-0.000741,0.005108,-0.001083,-0.000827,-0.003343,-0.005792,0.000594,0.013286,0.008056,-0.001489,...,0.015811,0.000359,-0.000096,-0.002604,-0.003749,0.001078,-0.001127,-0.000536,-0.001112,0.007452
2020-12-03,-0.002100,-0.009236,0.001747,-0.001490,-0.004473,0.001593,0.003165,-0.008446,0.003151,0.013005,...,0.007594,0.001537,-0.006268,-0.003549,-0.013740,0.005805,0.004052,0.000077,-0.001198,-0.000861
2020-12-04,0.000710,0.005997,0.009827,0.001990,0.005616,0.007816,0.008872,0.006972,0.013727,0.008586,...,0.013066,0.009491,-0.001386,0.002489,0.000605,0.023665,0.006218,0.005923,0.006303,0.011115
2020-12-07,-0.008122,0.010373,0.001582,-0.000331,-0.015637,-0.003246,-0.006449,0.003253,0.000202,0.000811,...,0.005122,0.007331,0.001951,-0.004867,0.005518,-0.000634,-0.005064,-0.003045,-0.002597,0.008785
2020-12-08,-0.001880,-0.003853,0.001656,-0.001656,0.004917,-0.002443,0.003147,-0.004842,0.011765,0.002430,...,-0.016212,0.002785,0.005023,0.009378,0.010284,0.014028,0.000042,0.002036,-0.001193,0.007281
2020-12-09,-0.011206,-0.002732,0.006859,-0.000829,0.004516,-0.003764,-0.000392,0.010080,0.003169,0.005091,...,0.020193,0.010166,0.013350,0.000478,0.005887,-0.008150,0.006172,0.003201,0.000893,0.002071


In [32]:
risk_dict = {}
for col in riskReturns:
    risk_dict[col] = []
    for i in range(9,len(riskReturns)):
        risk_dict[col].append(np.std(riskReturns[col].iloc[i-9 : i], ddof = 1))
risk_dict

{'000001.SS': [0.009150568643592307,
  0.008122769949883334,
  0.00834195743062761,
  0.0054625404913840195,
  0.0054708672089546625,
  0.005538157275906425,
  0.007163470207205757],
 '0P0000KSP8.L': [0.007689527530374342,
  0.007900360242318003,
  0.0073813311431250705,
  0.00703540790265264,
  0.006889895582616524,
  0.005869759764490769,
  0.005646896836586482],
 '0P0001GY56.F': [0.00928051180802401,
  0.00953276821696294,
  0.004816817008835314,
  0.005274610468335492,
  0.005625305831346778,
  0.006185308362828295,
  0.00539637168473606],
 'DAX': [0.007545003337581008,
  0.007333561379221314,
  0.00913491500815402,
  0.007491517839340014,
  0.009059530928509895,
  0.01000803748424854,
  0.010622471390816448],
 'ENY=F': [0.013652672944177944,
  0.012234533652455233,
  0.010730096677388615,
  0.0071000739233023375,
  0.007207297148422729,
  0.007033159323751011,
  0.006736956711071208],
 'FTSEMIB.MI': [0.0064707311905422915,
  0.005745146272730252,
  0.005035682235244856,
  0.005124

In [33]:
risks = pd.DataFrame(risk_dict)
risks

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
0,0.009151,0.007690,0.009281,0.007545,0.013653,0.006471,0.010474,0.009119,0.007733,0.007371,...,0.013446,0.004854,0.006254,0.005666,0.010085,0.012380,0.007342,0.005090,0.005688,0.007052
1,0.008123,0.007900,0.009533,0.007334,0.012235,0.005745,0.010525,0.009915,0.007444,0.007778,...,0.013746,0.005264,0.006446,0.006395,0.008028,0.012613,0.007596,0.005221,0.005503,0.008474
2,0.008342,0.007381,0.004817,0.009135,0.010730,0.005036,0.009245,0.008104,0.007648,0.006311,...,0.011338,0.005064,0.005788,0.006108,0.008083,0.010075,0.004567,0.004746,0.005584,0.006929
3,0.005463,0.007035,0.005275,0.007492,0.007100,0.005124,0.004789,0.008600,0.007285,0.005805,...,0.011243,0.004501,0.006216,0.005859,0.008363,0.010082,0.006138,0.004440,0.004812,0.006731
4,0.005471,0.006890,0.005625,0.009060,0.007207,0.005780,0.006545,0.007973,0.007362,0.005520,...,0.010721,0.004469,0.006556,0.005872,0.008316,0.011806,0.006105,0.004511,0.005160,0.007619
5,0.005538,0.005870,0.006185,0.010008,0.007033,0.005815,0.006632,0.007023,0.007362,0.004581,...,0.010639,0.004624,0.006188,0.005901,0.006575,0.012349,0.006199,0.005220,0.005508,0.009269
6,0.007163,0.005647,0.005396,0.010622,0.006737,0.005057,0.006481,0.006920,0.006452,0.004580,...,0.009996,0.004241,0.006153,0.005785,0.006448,0.010883,0.006384,0.004979,0.005365,0.008845


In [34]:
risks.index = volumes.index
risks

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-10,0.009151,0.007690,0.009281,0.007545,0.013653,0.006471,0.010474,0.009119,0.007733,0.007371,...,0.013446,0.004854,0.006254,0.005666,0.010085,0.012380,0.007342,0.005090,0.005688,0.007052
2020-12-11,0.008123,0.007900,0.009533,0.007334,0.012235,0.005745,0.010525,0.009915,0.007444,0.007778,...,0.013746,0.005264,0.006446,0.006395,0.008028,0.012613,0.007596,0.005221,0.005503,0.008474
2020-12-14,0.008342,0.007381,0.004817,0.009135,0.010730,0.005036,0.009245,0.008104,0.007648,0.006311,...,0.011338,0.005064,0.005788,0.006108,0.008083,0.010075,0.004567,0.004746,0.005584,0.006929
2020-12-15,0.005463,0.007035,0.005275,0.007492,0.007100,0.005124,0.004789,0.008600,0.007285,0.005805,...,0.011243,0.004501,0.006216,0.005859,0.008363,0.010082,0.006138,0.004440,0.004812,0.006731
2020-12-16,0.005471,0.006890,0.005625,0.009060,0.007207,0.005780,0.006545,0.007973,0.007362,0.005520,...,0.010721,0.004469,0.006556,0.005872,0.008316,0.011806,0.006105,0.004511,0.005160,0.007619
2020-12-17,0.005538,0.005870,0.006185,0.010008,0.007033,0.005815,0.006632,0.007023,0.007362,0.004581,...,0.010639,0.004624,0.006188,0.005901,0.006575,0.012349,0.006199,0.005220,0.005508,0.009269
2020-12-18,0.007163,0.005647,0.005396,0.010622,0.006737,0.005057,0.006481,0.006920,0.006452,0.004580,...,0.009996,0.004241,0.006153,0.005785,0.006448,0.010883,0.006384,0.004979,0.005365,0.008845


In [35]:
reg =pd.DataFrame(risks.mean(axis = 1), columns = ['risk'])
reg

,risk
Date,
2020-12-10,0.008311
2020-12-11,0.008273
2020-12-14,0.006903
2020-12-15,0.006107
2020-12-16,0.006389
2020-12-17,0.006479
2020-12-18,0.006322


In [36]:
reg['sentiment'] = sentiment_list
reg

,risk,sentiment
Date,,
2020-12-10,0.008311,0.048462
2020-12-11,0.008273,0.336568
2020-12-14,0.006903,0.061793
2020-12-15,0.006107,0.450239
2020-12-16,0.006389,0.404783
2020-12-17,0.006479,-0.049463
2020-12-18,0.006322,-0.061663


In [37]:
reg['size'] = volumes['size']
reg

,risk,sentiment,size
Date,,,
2020-12-10,0.008311,0.048462,0.202008
2020-12-11,0.008273,0.336568,0.201714
2020-12-14,0.006903,0.061793,0.201145
2020-12-15,0.006107,0.450239,0.201545
2020-12-16,0.006389,0.404783,0.200827
2020-12-17,0.006479,-0.049463,0.200510
2020-12-18,0.006322,-0.061663,0.204025


In [38]:
reg ['AAR'] = all_AR['AAR']
reg

,risk,sentiment,size,AAR
Date,,,,
2020-12-10,0.008311,0.048462,0.202008,0.000032
2020-12-11,0.008273,0.336568,0.201714,0.000023
2020-12-14,0.006903,0.061793,0.201145,0.000060
2020-12-15,0.006107,0.450239,0.201545,0.000045
2020-12-16,0.006389,0.404783,0.200827,-0.000035
2020-12-17,0.006479,-0.049463,0.200510,0.000002
2020-12-18,0.006322,-0.061663,0.204025,0.000041


In [39]:
reg1 = reg.iloc[0:2]
reg2 = reg.iloc[1:3]
reg3 = reg.iloc[2:4]
reg4 = reg.iloc[3:5]
reg5 = reg.iloc[4:6]
reg6 = reg.iloc[5:7]

In [40]:
import statsmodels.formula.api as smf

In [41]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                 -0.535
Method:                 Least Squares   F-statistic:                    0.3026
Date:                Mon, 07 Aug 2023   Prob (F-statistic):              0.824
Time:                        21:08:18   Log-Likelihood:                 64.001
No. Observations:                   7   AIC:                            -120.0
Df Residuals:                       3   BIC:                            -120.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0021      0.003     -0.711      0.5

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 7 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [42]:
# Conduct the Breusch-Pagan test
names = ['Lagrange multiplier statistic', 'p-value',
         'f-value', 'f p-value']
 
# Get the test result
test_result = sms.het_breuschpagan(fit.resid, fit.model.exog)
 
lzip(names, test_result)

[('Lagrange multiplier statistic', 3.781220633702364),
 ('p-value', 0.2860780555064432),
 ('f-value', 1.1747374403147333),
 ('f p-value', 0.4489034873656882)]

In [43]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg1).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 07 Aug 2023   Prob (F-statistic):                nan
Time:                        21:08:18   Log-Likelihood:                 89.128
No. Observations:                   2   AIC:                            -174.3
Df Residuals:                       0   BIC:                            -176.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.249e-05        inf          0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [44]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg2).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 07 Aug 2023   Prob (F-statistic):                nan
Time:                        21:08:18   Log-Likelihood:                 87.847
No. Observations:                   2   AIC:                            -171.7
Df Residuals:                       0   BIC:                            -174.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   6.588e-05        inf          0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [45]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg3).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 07 Aug 2023   Prob (F-statistic):                nan
Time:                        21:08:18   Log-Likelihood:                 90.737
No. Observations:                   2   AIC:                            -177.5
Df Residuals:                       0   BIC:                            -180.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   6.028e-05        inf          0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [46]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg4).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 07 Aug 2023   Prob (F-statistic):                nan
Time:                        21:08:18   Log-Likelihood:                 83.882
No. Observations:                   2   AIC:                            -163.8
Df Residuals:                       0   BIC:                            -166.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0007        inf         -0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [47]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg5).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 07 Aug 2023   Prob (F-statistic):                nan
Time:                        21:08:18   Log-Likelihood:                 92.123
No. Observations:                   2   AIC:                            -180.2
Df Residuals:                       0   BIC:                            -182.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.683e-06        inf         -0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [48]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg6).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 07 Aug 2023   Prob (F-statistic):                nan
Time:                        21:08:18   Log-Likelihood:                 86.330
No. Observations:                   2   AIC:                            -168.7
Df Residuals:                       0   BIC:                            -171.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0003        inf         -0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
